In [1]:
import requests
import json
import pandas as pd
import os
import time
import re
import ast

In [2]:
games = pd.read_csv('../data/rawg_games_raw_data.csv')
developers = pd.read_csv('../data/rawg_developer_raw_data.csv')
tags = pd.read_csv('../data/rawg_gametags_raw_data.csv')

## Cleaning Games DF

In [4]:
games

,Unnamed: 0,id,slug,name,released,tba,background_image,rating,rating_top,ratings,...,dominant_color,platforms,parent_platforms,genres,stores,clip,tags,esrb_rating,short_screenshots,community_rating
0,0,3498,grand-theft-auto-v,Grand Theft Auto V,2013-09-17,False,https://media.rawg.io/media/games/20a/20aa03a1...,4.47,5,"[{'id': 5, 'title': 'exceptional', 'count': 42...",...,0f0f0f,"[{'platform': {'id': 4, 'name': 'PC', 'slug': ...","[{'platform': {'id': 1, 'name': 'PC', 'slug': ...","[{'id': 4, 'name': 'Action', 'slug': 'action',...","[{'id': 290376, 'store': {'id': 1, 'name': 'St...",NaN,"[{'id': 31, 'name': 'Singleplayer', 'slug': 's...","{'id': 4, 'name': 'Mature', 'slug': 'mature'}","[{'id': -1, 'image': 'https://media.rawg.io/me...",NaN
1,1,3328,the-witcher-3-wild-hunt,The Witcher 3: Wild Hunt,2015-05-18,False,https://media.rawg.io/media/games/618/618c2031...,4.64,5,"[{'id': 5, 'title': 'exceptional', 'count': 53...",...,0f0f0f,"[{'platform': {'id': 186, 'name': 'Xbox Series...","[{'platform': {'id': 1, 'name': 'PC', 'slug': ...","[{'id': 4, 'name': 'Action', 'slug': 'action',...","[{'id': 354780, 'store': {'id': 5, 'name': 'GO...",NaN,"[{'id': 31, 'name': 'Singleplayer', 'slug': 's...","{'id': 4, 'name': 'Mature', 'slug': 'mature'}","[{'id': -1, 'image': 'https://media.rawg.io/me...",NaN
2,2,4200,portal-2,Portal 2,2011-04-18,False,https://media.rawg.io/media/games/2ba/2bac0e87...,4.59,5,"[{'id': 5, 'title': 'exceptional', 'count': 41...",...,0f0f0f,"[{'platform': {'id': 16, 'name': 'PlayStation ...","[{'platform': {'id': 1, 'name': 'PC', 'slug': ...","[{'id': 2, 'name': 'Shooter', 'slug': 'shooter...","[{'id': 465889, 'store': {'id': 2, 'name': 'Xb...",NaN,"[{'id': 31, 'name': 'Singleplayer', 'slug': 's...","{'id': 2, 'name': 'Everyone 10+', 'slug': 'eve...","[{'id': -1, 'image': 'https://media.rawg.io/me...",NaN
3,3,4291,counter-strike-global-offensive,Counter-Strike: Global Offensive,2012-08-21,False,https://media.rawg.io/media/games/736/73619bd3...,3.57,4,"[{'id': 4, 'title': 'recommended', 'count': 16...",...,0f0f0f,"[{'platform': {'id': 4, 'name': 'PC', 'slug': ...","[{'platform': {'id': 1, 'name': 'PC', 'slug': ...","[{'id': 2, 'name': 'Shooter', 'slug': 'shooter...","[{'id': 4619, 'store': {'id': 3, 'name': 'Play...",NaN,"[{'id': 40847, 'name': 'Steam Achievements', '...","{'id': 4, 'name': 'Mature', 'slug': 'mature'}","[{'id': -1, 'image': 'https://media.rawg.io/me...",NaN
4,4,5286,tomb-raider,Tomb Raider (2013),2013-03-05,False,https://media.rawg.io/media/games/021/021c4e21...,4.06,4,"[{'id': 4, 'title': 'recommended', 'count': 24...",...,0f0f0f,"[{'platform': {'id': 16, 'name': 'PlayStation ...","[{'platform': {'id': 1, 'name': 'PC', 'slug': ...","[{'id': 4, 'name': 'Action', 'slug': 'action',...","[{'id': 33824, 'store': {'id': 7, 'name': 'Xbo...",NaN,"[{'id': 31, 'name': 'Singleplayer', 'slug': 's...","{'id': 4, 'name': 'Mature', 'slug': 'mature'}","[{'id': -1, 'image': 'https://media.rawg.io/me...",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,19995,21970,crimson-earth,Crimson Earth,2017-05-09,False,https://media.rawg.io/media/screenshots/507/50...,0.00,0,"[{'id': 1, 'title': 'skip', 'count': 2, 'perce...",...,0f0f0f,"[{'platform': {'id': 4, 'name': 'PC', 'slug': ...","[{'platform': {'id': 1, 'name': 'PC', 'slug': ...","[{'id': 4, 'name': 'Action', 'slug': 'action',...","[{'id': 24731, 'store': {'id': 1, 'name': 'Ste...",NaN,"[{'id': 31, 'name': 'Singleplayer', 'slug': 's...",NaN,"[{'id': -1, 'image': 'https://media.rawg.io/me...",0.0
19996,19996,21553,slamit-pinball-big-score,SlamIt Pinball Big Score,2009-02-23,False,https://media.rawg.io/media/screenshots/ad7/ad...,0.00,0,"[{'id': 4, 'title': 'recommended', 'count': 1,...",...,0f0f0f,"[{'platform': {'id': 4, 'name': 'PC', 'slug': ...","[{'platform': {'id': 1, 'name': 'PC', 'slug': ...","[{'id': 11, 'name': 'Arcade', 'slug': 'arcade'...","[{'id': 24251, 'store': {'id': 1, 'name': 'Ste...",NaN,"[{'id': 31, 'n

In [5]:
def remove_string(list_str):
        extracted_list = ast.literal_eval(list_str)
        return extracted_list

In [6]:
games = games.drop(columns = ['background_image', 'saturated_color', 'dominant_color', 'short_screenshots', 'clip'])
games.head(2)

,Unnamed: 0,id,slug,name,released,tba,rating,rating_top,ratings,ratings_count,...,updated,user_game,reviews_count,platforms,parent_platforms,genres,stores,tags,esrb_rating,community_rating
0,0,3498,grand-theft-auto-v,Grand Theft Auto V,2013-09-17,False,4.47,5,"[{'id': 5, 'title': 'exceptional', 'count': 42...",7118,...,2025-06-10T15:19:04,NaN,7232,"[{'platform': {'id': 4, 'name': 'PC', 'slug': ...","[{'platform': {'id': 1, 'name': 'PC', 'slug': ...","[{'id': 4, 'name': 'Action', 'slug': 'action',...","[{'id': 290376, 'store': {'id': 1, 'name': 'St...","[{'id': 31, 'name': 'Singleplayer', 'slug': 's...","{'id': 4, 'name': 'Mature', 'slug': 'mature'}",NaN
1,1,3328,the-witcher-3-wild-hunt,The Witcher 3: Wild Hunt,2015-05-18,False,4.64,5,"[{'id': 5, 'title': 'exceptional', 'count': 53...",6913,...,2025-06-10T15:17:24,NaN,7026,"[{'platform': {'id': 186, 'name': 'Xbox Series...","[{'platform': {'id': 1, 'name': 'PC', 'slug': ...","[{'id': 4, 'name': 'Action', 'slug': 'action',...","[{'id': 354780, 'store': {'id': 5, 'name': 'GO...","[{'id': 31, 'name': 'Singleplayer', 'slug': 's...","{'id': 4, 'name': 'Mature', 'slug': 'mature'}",NaN


In [7]:
json_columns = ['ratings', 'stores', 'parent_platforms', 'tags', 'platforms', 'added_by_status', 'genres']

for i in games.columns:
    if i in json_columns:
        games[i] = games[i].apply(remove_string)
        games = games[~games[i].apply(lambda x: x == [])]

In [8]:
games = games.reset_index(drop=True)
games_1 = games

In [9]:
games_1.columns

Index(['Unnamed: 0', 'id', 'slug', 'name', 'released', 'tba', 'rating',
       'rating_top', 'ratings', 'ratings_count', 'reviews_text_count', 'added',
       'added_by_status', 'metacritic', 'playtime', 'suggestions_count',
       'updated', 'user_game', 'reviews_count', 'platforms',
       'parent_platforms', 'genres', 'stores', 'tags', 'esrb_rating',
       'community_rating'],
      dtype='object')

In [10]:
pd.DataFrame(pd.json_normalize(games_1['ratings'])[0][0], index=[0])

,id,title,count,percent
0,5,exceptional,4267,59.0


In [11]:
pd.json_normalize(games['ratings'][2]).set_index('title', drop=True)[['percent']].T.reset_index()

title,index,exceptional,recommended,meh,skip
0,percent,69.33,25.16,3.02,2.5


<hr>

<h3>Cleaning Ratings Column</h3>

In [14]:
concat_list = []

for i in range(len(games['ratings'])):
    concat_list.append(
        pd.json_normalize(
            games['ratings'][i]
        ).set_index('title', drop=True)[['percent']].T
    )
    print(f'finished at index {i}')
ratings_df = pd.concat(concat_list)

renamed = {'exceptional':'rating_exceptional','recommended':'rating_recommended','meh':'rating_meh','skip':'rating_skip'}
ratings_df = ratings_df.rename(columns=renamed).reset_index(drop=True)

games = pd.concat([games, ratings_df], axis=1).drop(columns=['ratings'])

finished at index 0
finished at index 1
finished at index 2
finished at index 3
finished at index 4
finished at index 5
finished at index 6
finished at index 7
finished at index 8
finished at index 9
finished at index 10
finished at index 11
finished at index 12
finished at index 13
finished at index 14
finished at index 15
finished at index 16
finished at index 17
finished at index 18
finished at index 19
finished at index 20
finished at index 21
finished at index 22
finished at index 23
finished at index 24
finished at index 25
finished at index 26
finished at index 27
finished at index 28
finished at index 29
finished at index 30
finished at index 31
finished at index 32
finished at index 33
finished at index 34
finished at index 35
finished at index 36
finished at index 37
finished at index 38
finished at index 39
finished at index 40
finished at index 41
finished at index 42
finished at index 43
finished at index 44
finished at index 45
finished at index 46
finished at index 47
fi

<hr>

<h3>Cleaning Added Column</h3>

In [17]:
games['added_by_status']

0        {'yet': 557, 'owned': 12714, 'beaten': 6254, '...
1        {'yet': 1175, 'owned': 12714, 'beaten': 5091, ...
2        {'yet': 658, 'owned': 13016, 'beaten': 5716, '...
3        {'yet': 267, 'owned': 14084, 'beaten': 1109, '...
4        {'yet': 694, 'owned': 11545, 'beaten': 4402, '...
                               ...                        
17281    {'yet': 4, 'owned': 46, 'beaten': 3, 'toplay':...
17282    {'yet': 1, 'owned': 54, 'beaten': 2, 'dropped'...
17283    {'yet': 1, 'owned': 56, 'beaten': 1, 'dropped'...
17284    {'owned': 51, 'beaten': 1, 'toplay': 2, 'dropp...
17285                                        {'owned': 59}
Name: added_by_status, Length: 17286, dtype: object

In [18]:
pd.json_normalize(games['added_by_status'][1])

,yet,owned,beaten,toplay,dropped,playing
0,1175,12714,5091,834,995,893


In [19]:
added_list = []

for i in range(len(games['added_by_status'])):
    if len(games['added_by_status'][i]) != 0:
        json_norm =  pd.json_normalize(games['added_by_status'][i])
        added_list.append(json_norm)
    else:
        added_list.append(i)
        print(f'index {i} complete')

added_df = pd.concat(added_list)

In [20]:
added_df = added_df.drop(columns=['yet', 'toplay']).reset_index(drop=True)

In [21]:
added_df

,owned,beaten,dropped,playing
0,12714,6254.0,1154.0,753.0
1,12714,5091.0,995.0,893.0
2,13016,5716.0,627.0,161.0
3,14084,1109.0,2097.0,631.0
4,11545,4402.0,558.0,116.0
...,...,...,...,...
17281,46,3.0,3.0,NaN
17282,54,2.0,2.0,NaN
17283,56,1.0,1.0,NaN
17284,51,1.0,4.0,1.0


In [22]:
print(added_df.shape)
print(games.shape)

(17286, 4)
(17286, 29)


In [23]:
games = pd.concat([games, added_df], axis=1).drop(columns=['added_by_status'])

<hr>

<h3>Cleaning ESRB Rating Column</h3>

In [26]:
games['esrb_rating'] = games['esrb_rating'].apply(lambda x: remove_string(x) if pd.notna(x) else x)

In [27]:
games['esrb_rating'] = pd.json_normalize(games['esrb_rating'])[['name']].rename(columns={'name':'esrb_rating'})

<h4>Exporting ESRB List</h4>

In [29]:
games_1['esrb_rating'] = games_1['esrb_rating'].apply(lambda x: remove_string(x) if pd.notna(x) else x)

In [30]:
pd.json_normalize(games_1['esrb_rating'])['name'].unique()

array(['Mature', 'Everyone 10+', 'Teen', 'Adults Only', nan, 'Everyone',
       'Rating Pending'], dtype=object)

<hr>

<h3>Cleaning Genres Column</h3>

In [33]:
games['genres']

0        [{'id': 4, 'name': 'Action', 'slug': 'action',...
1        [{'id': 4, 'name': 'Action', 'slug': 'action',...
2        [{'id': 2, 'name': 'Shooter', 'slug': 'shooter...
3        [{'id': 2, 'name': 'Shooter', 'slug': 'shooter...
4        [{'id': 4, 'name': 'Action', 'slug': 'action',...
                               ...                        
17281    [{'id': 4, 'name': 'Action', 'slug': 'action',...
17282    [{'id': 4, 'name': 'Action', 'slug': 'action',...
17283    [{'id': 11, 'name': 'Arcade', 'slug': 'arcade'...
17284    [{'id': 14, 'name': 'Simulation', 'slug': 'sim...
17285    [{'id': 10, 'name': 'Strategy', 'slug': 'strat...
Name: genres, Length: 17286, dtype: object

In [34]:
pd.json_normalize(games['genres'][1])

,id,name,slug,games_count,image_background
0,4,Action,action,187659,https://media.rawg.io/media/games/49c/49c3dfa4...
1,5,RPG,role-playing-games-rpg,59905,https://media.rawg.io/media/games/3cf/3cff8999...


In [35]:
genre_list = []

for i in range(len(games['genres'])):
    if len(games['genres'][i]) != 0:
        json_norm =  pd.json_normalize(games['genres'][i])
        plats = pd.DataFrame({x: (x in json_norm['name'].values) for x in json_norm['name'] if x != []}, index=[i])
        print(f'index {i} complete')
        genre_list.append(plats)
    else:
        store_list.append(i)
        print(f'index {i} complete')

genre_df = pd.concat(genre_list)

index 0 complete
index 1 complete
index 2 complete
index 3 complete
index 4 complete
index 5 complete
index 6 complete
index 7 complete
index 8 complete
index 9 complete
index 10 complete
index 11 complete
index 12 complete
index 13 complete
index 14 complete
index 15 complete
index 16 complete
index 17 complete
index 18 complete
index 19 complete
index 20 complete
index 21 complete
index 22 complete
index 23 complete
index 24 complete
index 25 complete
index 26 complete
index 27 complete
index 28 complete
index 29 complete
index 30 complete
index 31 complete
index 32 complete
index 33 complete
index 34 complete
index 35 complete
index 36 complete
index 37 complete
index 38 complete
index 39 complete
index 40 complete
index 41 complete
index 42 complete
index 43 complete
index 44 complete
index 45 complete
index 46 complete
index 47 complete
index 48 complete
index 49 complete
index 50 complete
index 51 complete
index 52 complete
index 53 complete
index 54 complete
index 55 complete
in

In [36]:
print(genre_df.shape)
print(games.shape)

(17286, 19)
(17286, 32)


In [37]:
genre_df = genre_df.rename(columns=lambda x: x + '_genre')

In [38]:
games = pd.concat([games, genre_df], axis=1).drop(columns = ['genres'])

In [39]:
# outer_genre_names = []

# for j in range(len(games['genres'])):
#     inner_genre_names = []    
#     for i in games['genres'].iloc[j]:
#         inner_genre_names.append(i['name'])   
#     total_genre_name = '_'.join(inner_genre_names)
#     outer_genre_names.append(total_genre_name)

# games['genres'] = pd.DataFrame(outer_genre_names, columns = ['genre'])

<h4>Exporting Genre List</h4>

In [41]:
genre_list_1 = []
for i in range(len(games_1['genres'])):
    genre_list_1.append(pd.json_normalize(games_1['genres'][i]))

genre_1_df = pd.concat(genre_list_1)

In [42]:
genre_export_list = genre_1_df[['name']].reset_index(drop=True)['name'].unique()

In [43]:
pot_list = pd.DataFrame(genre_export_list).rename(columns={0:'name'})

In [44]:
list(pot_list['name'])

['Action',
 'RPG',
 'Shooter',
 'Puzzle',
 'Adventure',
 'Indie',
 'Platformer',
 'Massively Multiplayer',
 'Sports',
 'Racing',
 'Simulation',
 'Arcade',
 'Casual',
 'Strategy',
 'Fighting',
 'Family',
 'Educational',
 'Card',
 'Board Games']

In [45]:
pd.DataFrame(genre_export_list).rename(columns={0:'name'}).to_csv('../data/genre_list.csv')

<hr>

<h3>Cleaning Parent Platform Column</h3>

In [48]:
# platform_list = []

# for i in range(len(games['parent_platforms'])):
#     if len(games['parent_platforms'][i]) != 0:
#         json_norm =  pd.json_normalize(games['parent_platforms'][i])
#         plats = pd.DataFrame({x: (x in json_norm['platform.name'].values) for x in json_norm['platform.name'] if x != []}, index=[i])
#         print(f'index {i} complete')
#         platform_list.append(plats)
#     else:
#         platform_list.append(pd.json_normalize(games['parent_platforms'][i]))
#         print(f'index {i} complete')

# platform_df = pd.concat(platform_list)

In [49]:
# print(platform_df.shape)
# print(games.shape)

In [50]:
#platform_df = platform_df.drop(columns = ['SEGA','Commodore / Amiga','Neo Geo','Atari','3DO']).rename(columns=lambda x: x + '_platform')

In [51]:
#games = pd.concat([games, platform_df], axis=1).drop(columns=['parent_platforms'])

<hr>

<h3>Cleaning Storefront Column</h3>

In [54]:
pd.json_normalize(games['stores'][0])

,id,store.id,store.name,store.slug,store.domain,store.games_count,store.image_background
0,290376,1,Steam,steam,store.steampowered.com,113018,https://media.rawg.io/media/games/2ba/2bac0e87...
1,290375,3,PlayStation Store,playstation-store,store.playstation.com,8051,https://media.rawg.io/media/games/4cf/4cfc6b7f...
2,438095,11,Epic Games,epic-games,epicgames.com,1400,https://media.rawg.io/media/games/5c0/5c0dd630...
3,290377,7,Xbox 360 Store,xbox360,marketplace.xbox.com,1915,https://media.rawg.io/media/games/021/021c4e21...
4,290378,2,Xbox Store,xbox-store,microsoft.com,4928,https://media.rawg.io/media/games/587/587588c6...


In [55]:
store_list = []

for i in range(len(games['stores'])):
    if len(games['stores'][i]) != 0:
        json_norm =  pd.json_normalize(games['stores'][i])
        plats = pd.DataFrame({x: (x in json_norm['store.name'].values) for x in json_norm['store.name'] if x != []}, index=[i])
        print(f'index {i} complete')
        store_list.append(plats)
    else:
        store_list.append(i)
        print(f'index {i} complete')

store_df = pd.concat(store_list)

index 0 complete
index 1 complete
index 2 complete
index 3 complete
index 4 complete
index 5 complete
index 6 complete
index 7 complete
index 8 complete
index 9 complete
index 10 complete
index 11 complete
index 12 complete
index 13 complete
index 14 complete
index 15 complete
index 16 complete
index 17 complete
index 18 complete
index 19 complete
index 20 complete
index 21 complete
index 22 complete
index 23 complete
index 24 complete
index 25 complete
index 26 complete
index 27 complete
index 28 complete
index 29 complete
index 30 complete
index 31 complete
index 32 complete
index 33 complete
index 34 complete
index 35 complete
index 36 complete
index 37 complete
index 38 complete
index 39 complete
index 40 complete
index 41 complete
index 42 complete
index 43 complete
index 44 complete
index 45 complete
index 46 complete
index 47 complete
index 48 complete
index 49 complete
index 50 complete
index 51 complete
index 52 complete
index 53 complete
index 54 complete
index 55 complete
in

In [56]:
print(store_df.shape)
print(games.shape)

(17286, 10)
(17286, 50)


In [57]:
store_df = store_df.drop(columns = ['Xbox 360 Store']).rename(columns=lambda x: x + '_store')
games = pd.concat([games, store_df], axis=1).drop(columns=['stores'])

<h4>Exporting Stores List</h4>

In [59]:
stores_list_1 = []
for i in range(len(games_1['stores'])):
    stores_list_1.append(pd.json_normalize(games_1['stores'][i]))

stores_1_df = pd.concat(stores_list_1)

stores_export_list = stores_1_df[['store.name']].reset_index(drop=True)['store.name'].unique()

In [60]:
stores_export_list

array(['Steam', 'PlayStation Store', 'Epic Games', 'Xbox 360 Store',
       'Xbox Store', 'GOG', 'Nintendo Store', 'Google Play', 'App Store',
       'itch.io'], dtype=object)

In [61]:
pd.DataFrame(stores_export_list).rename(columns={0:'name'}).to_csv('../data/stores_list.csv')

<hr>

<h3>Cleaning Tags Column</h3>

In [64]:
pd.json_normalize(games['tags'].iloc[0])

,id,name,slug,language,games_count,image_background
0,31,Singleplayer,singleplayer,eng,241108,https://media.rawg.io/media/games/b45/b45575f3...
1,40847,Steam Achievements,steam-achievements,eng,46424,https://media.rawg.io/media/games/9fa/9fa63622...
2,7,Multiplayer,multiplayer,eng,40923,https://media.rawg.io/media/games/021/021c4e21...
3,40836,Full controller support,full-controller-support,eng,21864,https://media.rawg.io/media/games/f46/f466571d...
4,13,Atmospheric,atmospheric,eng,37360,https://media.rawg.io/media/games/6cd/6cd653e0...
5,42,Great Soundtrack,great-soundtrack,eng,3429,https://media.rawg.io/media/games/b7b/b7b83817...
6,24,RPG,rpg,eng,24637,https://media.rawg.io/media/games/15c/15c95a49...
7,18,Co-op,co-op,eng,13373,https://media.rawg.io/media/games/4a0/4a0a1316...
8,36,Open World,open-world,eng,8668,https://media.rawg.io/media/games/49c/49c3dfa4...
9,411,cooperative,cooperative,eng,6009,https://media.rawg.io/media/games/d0f/d0f91fe1...


In [65]:
rawg_tag_list = list(tags['name'][:50])
tag_list = []

for i in range(len(games['tags'])):
    if len(games['tags'][i]) != 0:
        json_norm =  pd.json_normalize(games['tags'][i])
        plats = pd.DataFrame({x: (x in json_norm['name'].values) for x in json_norm['name'] if x != [] and x in rawg_tag_list}, index=[i])
        print(f'index {i} complete')
        tag_list.append(plats)
    else:
        tag_list.append(i)
        print(f'index {i} complete')

tag_df = pd.concat(tag_list)

index 0 complete
index 1 complete
index 2 complete
index 3 complete
index 4 complete
index 5 complete
index 6 complete
index 7 complete
index 8 complete
index 9 complete
index 10 complete
index 11 complete
index 12 complete
index 13 complete
index 14 complete
index 15 complete
index 16 complete
index 17 complete
index 18 complete
index 19 complete
index 20 complete
index 21 complete
index 22 complete
index 23 complete
index 24 complete
index 25 complete
index 26 complete
index 27 complete
index 28 complete
index 29 complete
index 30 complete
index 31 complete
index 32 complete
index 33 complete
index 34 complete
index 35 complete
index 36 complete
index 37 complete
index 38 complete
index 39 complete
index 40 complete
index 41 complete
index 42 complete
index 43 complete
index 44 complete
index 45 complete
index 46 complete
index 47 complete
index 48 complete
index 49 complete
index 50 complete
index 51 complete
index 52 complete
index 53 complete
index 54 complete
index 55 complete
in

In [66]:
print(tag_df.shape)
print(games.shape)

(17286, 50)
(17286, 58)


In [67]:
games = pd.concat([games, tag_df], axis=1).drop(columns=['tags'])

<h4>Exporting Tags List</h4>

In [69]:
tag_list_1 = []
for i in range(len(games_1['tags'])):
    tag_list_1.append(pd.json_normalize(games_1['tags'][i]))

tag_1_df = pd.concat(tag_list_1)

tag_list = tag_1_df[['name']].reset_index(drop=True)['name'].unique()

In [70]:
pd.DataFrame(stores_export_list).rename(columns={0:'name'}).to_csv('../data/tag_list.csv')

<hr>

<h3>Cleaning Platforms Column</h3>

In [73]:
platform_list = []

for i in range(len(games['platforms'])):
    if len(games['platforms'][i]) != 0:
        json_norm =  pd.json_normalize(games['platforms'][i])
        plats = pd.DataFrame({x: (x in json_norm['platform.name'].values) for x in json_norm['platform.name'] if x != []}, index=[i])
        print(f'index {i} complete')
        platform_list.append(plats)
    else:
        store_list.append(i)
        print(f'index {i} complete')

platform_df = pd.concat(platform_list)

index 0 complete
index 1 complete
index 2 complete
index 3 complete
index 4 complete
index 5 complete
index 6 complete
index 7 complete
index 8 complete
index 9 complete
index 10 complete
index 11 complete
index 12 complete
index 13 complete
index 14 complete
index 15 complete
index 16 complete
index 17 complete
index 18 complete
index 19 complete
index 20 complete
index 21 complete
index 22 complete
index 23 complete
index 24 complete
index 25 complete
index 26 complete
index 27 complete
index 28 complete
index 29 complete
index 30 complete
index 31 complete
index 32 complete
index 33 complete
index 34 complete
index 35 complete
index 36 complete
index 37 complete
index 38 complete
index 39 complete
index 40 complete
index 41 complete
index 42 complete
index 43 complete
index 44 complete
index 45 complete
index 46 complete
index 47 complete
index 48 complete
index 49 complete
index 50 complete
index 51 complete
index 52 complete
index 53 complete
index 54 complete
index 55 complete
in

In [74]:
print(platform_df.shape)
print(games.shape)

(17286, 50)
(17286, 107)


In [75]:
platform_df = platform_df.iloc[:,:14].drop(columns=['Xbox 360', 'Xbox', 'PS Vita', 'PlayStation 3']).rename(columns=lambda x: x + '_platform')

In [76]:
games = pd.concat([games, platform_df], axis=1).drop(columns=['platforms'])

In [77]:
games.iloc[:, 28:] = games.iloc[:, 28:].notna().astype(int)

<h4>Exporting Platform List</h4>

In [79]:
pd.json_normalize(games_1['platforms'][0])

,released_at,requirements_ru,platform.id,platform.name,platform.slug,platform.image,platform.year_end,platform.year_start,platform.games_count,platform.image_background,requirements_en.minimum,requirements_en.recommended,requirements_en
0,2013-09-17,None,4,PC,pc,None,None,NaN,550071,https://media.rawg.io/media/games/4be/4be6a6ad...,"Minimum:OS: Windows 10 64 Bit, Windows 8.1 64 ...","Recommended:OS: Windows 10 64 Bit, Windows 8.1...",NaN
1,2013-09-17,None,187,PlayStation 5,playstation5,None,None,2020.0,1276,https://media.rawg.io/media/games/e74/e7445805...,NaN,NaN,NaN
2,2013-09-17,None,186,Xbox Series S/X,xbox-series-x,None,None,2020.0,1102,https://media.rawg.io/media/games/ffe/ffed8710...,NaN,NaN,NaN
3,2013-09-17,None,18,PlayStation 4,playstation4,None,None,NaN,6925,https://media.rawg.io/media/games/34b/34b1f185...,NaN,NaN,NaN
4,2013-09-17,None,16,PlayStation 3,playstation3,None,None,NaN,3166,https://media.rawg.io/media/games/d1a/d1a2e99a...,NaN,NaN,NaN
5,2013-09-17,None,14,Xbox 360,xbox360,None,None,NaN,2805,https://media.rawg.io/media/games/490/49016e06...,NaN,NaN,NaN
6,2013-09-17,None,1,Xbox One,xbox-one,None,None,NaN,5700,https://media.rawg.io/media/games/73e/73eecb89...,NaN,NaN,NaN


In [80]:
platforms_list_1 = []
for i in range(len(games_1['platforms'])):
    platforms_list_1.append(pd.json_normalize(games_1['platforms'][i]))

platforms_1_df = pd.concat(platforms_list_1)

platforms_list = platforms_1_df[['platform.name']].reset_index(drop=True)['platform.name'].unique()
platforms_list

C:\Users\Gubbz\AppData\Local\Temp\ipykernel_326948\517019770.py:5: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  platforms_1_df = pd.concat(platforms_list_1)


array(['PC', 'PlayStation 5', 'Xbox Series S/X', 'PlayStation 4',
       'PlayStation 3', 'Xbox 360', 'Xbox One', 'macOS',
       'Nintendo Switch', 'Linux', 'Android', 'Xbox', 'PS Vita', 'iOS',
       'Web', 'Wii U', 'Nintendo 3DS', 'PlayStation 2', 'Dreamcast',
       'Classic Macintosh', 'GameCube', 'Wii', 'Nintendo DS',
       'Nintendo 64', 'PlayStation', 'SEGA Saturn', 'Game Boy Advance',
       'PSP', 'Nintendo DSi', 'Commodore / Amiga', 'Game Boy Color',
       'Neo Geo', 'SEGA CD', 'Apple II', 'Atari ST', 'SEGA Master System',
       'Atari 8-bit', 'NES', 'Game Boy', 'SNES', 'Jaguar', 'Genesis',
       '3DO', 'Game Gear', 'SEGA 32X', 'Atari Lynx', 'Atari 5200',
       'Atari 2600', 'Atari 7800', 'Atari XEGS'], dtype=object)

In [81]:
pd.DataFrame(stores_export_list).rename(columns={0:'name'}).to_csv('../data/platforms_list.csv')

<hr>

## Cleaning Developers

In [84]:
developers

,Unnamed: 0,id,name,slug,games_count,image_background,games
0,0,1612,Valve Software,valve-software,44,https://media.rawg.io/media/games/46d/46d98e69...,"[{'id': 4200, 'slug': 'portal-2', 'name': 'Por..."
1,1,405,Ubisoft,ubisoft,346,https://media.rawg.io/media/games/2ee/2eef5ed5...,"[{'id': 4161, 'slug': 'far-cry-3', 'name': 'Fa..."
2,2,18893,Feral Interactive,feral-interactive,107,https://media.rawg.io/media/screenshots/375/37...,"[{'id': 7689, 'slug': 'rise-of-the-tomb-raider..."
3,3,3709,Ubisoft Montreal,ubisoft-montreal,121,https://media.rawg.io/media/games/99b/99b39612...,"[{'id': 4161, 'slug': 'far-cry-3', 'name': 'Fa..."
4,4,4132,Square Enix,square-enix,368,https://media.rawg.io/media/games/214/2140885d...,"[{'id': 10035, 'slug': 'hitman', 'name': 'Hitm..."
...,...,...,...,...,...,...,...
14995,14995,106339,JiSheng Fu,jisheng-fu,3,https://media.rawg.io/media/screenshots/f2f/f2...,"[{'id': 44219, 'slug': 'escape-room', 'name': ..."
14996,14996,104515,"Bowser Jr, Larry & Dottie",bowser-jr-larry-dottie,4,https://media.rawg.io/media/screenshots/eb8/eb...,"[{'id': 256822, 'slug': 'shark-dash', 'name': ..."
14997,14997,104020,Alkis Hadjinicolaou,alkis-hadjinicolaou,3,https://media.rawg.io/media/screenshots/0b0/0b...,"[{'id': 22450, 'slug': 'the-language-game', 'n..."
14998,14998,102714,ChaosForge,chaosforge,1,https://media.rawg.io/media/screenshots/c86/c8...,"[{'id': 250011, 'slug': 'jupiter-hell', 'name'..."


In [85]:
developers['games'] = developers['games'].apply(remove_string)
developers

,Unnamed: 0,id,name,slug,games_count,image_background,games
0,0,1612,Valve Software,valve-software,44,https://media.rawg.io/media/games/46d/46d98e69...,"[{'id': 4200, 'slug': 'portal-2', 'name': 'Por..."
1,1,405,Ubisoft,ubisoft,346,https://media.rawg.io/media/games/2ee/2eef5ed5...,"[{'id': 4161, 'slug': 'far-cry-3', 'name': 'Fa..."
2,2,18893,Feral Interactive,feral-interactive,107,https://media.rawg.io/media/screenshots/375/37...,"[{'id': 7689, 'slug': 'rise-of-the-tomb-raider..."
3,3,3709,Ubisoft Montreal,ubisoft-montreal,121,https://media.rawg.io/media/games/99b/99b39612...,"[{'id': 4161, 'slug': 'far-cry-3', 'name': 'Fa..."
4,4,4132,Square Enix,square-enix,368,https://media.rawg.io/media/games/214/2140885d...,"[{'id': 10035, 'slug': 'hitman', 'name': 'Hitm..."
...,...,...,...,...,...,...,...
14995,14995,106339,JiSheng Fu,jisheng-fu,3,https://media.rawg.io/media/screenshots/f2f/f2...,"[{'id': 44219, 'slug': 'escape-room', 'name': ..."
14996,14996,104515,"Bowser Jr, Larry & Dottie",bowser-jr-larry-dottie,4,https://media.rawg.io/media/screenshots/eb8/eb...,"[{'id': 256822, 'slug': 'shark-dash', 'name': ..."
14997,14997,104020,Alkis Hadjinicolaou,alkis-hadjinicolaou,3,https://media.rawg.io/media/screenshots/0b0/0b...,"[{'id': 22450, 'slug': 'the-language-game', 'n..."
14998,14998,102714,ChaosForge,chaosforge,1,https://media.rawg.io/media/screenshots/c86/c8...,"[{'id': 250011, 'slug': 'jupiter-hell', 'name'..."


In [86]:
l = []
for i in range(len(developers)):
    df = pd.json_normalize(developers['games'][i])
    df['developer'] = developers['name'][i]
    l.append(df)

developer_df = pd.concat(l).reset_index()
developer_df

,index,id,slug,name,added,developer
0,0,4200.0,portal-2,Portal 2,20576.0,Valve Software
1,1,4291.0,counter-strike-global-offensive,Counter-Strike: Global Offensive,18267.0,Valve Software
2,2,13536.0,portal,Portal,17541.0,Valve Software
3,3,12020.0,left-4-dead-2,Left 4 Dead 2,17379.0,Valve Software
4,4,13537.0,half-life-2,Half-Life 2,15858.0,Valve Software
...,...,...,...,...,...,...
48152,0,22450.0,the-language-game,The Language Game,41.0,Alkis Hadjinicolaou
48153,1,255285.0,the-great-movie-game,The Great Movie Game,0.0,Alkis Hadjinicolaou
48154,2,255286.0,ar-cubes,AR Cubes,0.0,Alkis Hadjinicolaou
48155,0,250011.0,jupiter-hell,Jupiter Hell,170.0,ChaosForge


## Combining Games with Developers

In [88]:
full_games_df = pd.merge(games, developer_df, how='left', on=['slug', 'name']).drop(columns = ['Unnamed: 0', 'id_x', 'id_y', 'added_y','index', 'added_x'])

In [89]:
full_games_df.columns

Index(['slug', 'name', 'released', 'tba', 'rating', 'rating_top',
       'ratings_count', 'reviews_text_count', 'metacritic', 'playtime',
       ...
       'PlayStation 5_platform', 'Xbox Series S/X_platform',
       'PlayStation 4_platform', 'Xbox One_platform', 'macOS_platform',
       'Nintendo Switch_platform', 'Linux_platform', 'Android_platform',
       'iOS_platform', 'developer'],
      dtype='object', length=114)

In [90]:
full_games_df.shape

(21868, 114)

In [91]:
#full_games_df.to_csv('../data/rawg_cleaned_games_data.csv')

## Adding VG Sales Data

In [93]:
columns_to_drop = ['img']

vg_sales = pd.read_csv('../data/vgchartz-2024.csv')
vg_sales = vg_sales.drop(columns = columns_to_drop)

In [94]:
vg_sales

,title,console,genre,publisher,developer,critic_score,total_sales,na_sales,jp_sales,pal_sales,other_sales,release_date,last_update
0,Grand Theft Auto V,PS3,Action,Rockstar Games,Rockstar North,9.4,20.32,6.37,0.99,9.85,3.12,2013-09-17,NaN
1,Grand Theft Auto V,PS4,Action,Rockstar Games,Rockstar North,9.7,19.39,6.06,0.60,9.71,3.02,2014-11-18,2018-01-03
2,Grand Theft Auto: Vice City,PS2,Action,Rockstar Games,Rockstar North,9.6,16.15,8.41,0.47,5.49,1.78,2002-10-28,NaN
3,Grand Theft Auto V,X360,Action,Rockstar Games,Rockstar North,NaN,15.86,9.06,0.06,5.33,1.42,2013-09-17,NaN
4,Call of Duty: Black Ops 3,PS4,Shooter,Activision,Treyarch,8.1,15.09,6.18,0.41,6.05,2.44,2015-11-06,2018-01-14
...,...,...,...,...,...,...,...,...,...,...,...,...,...
64011,XBlaze Lost: Memories,PC,Visual Novel,Aksys Games,Arc System Works,NaN,NaN,NaN,NaN,NaN,NaN,2016-08-11,2019-01-28
64012,"Yoru, Tomosu",PS4,Visual Novel,Nippon Ichi Software,Nippon Ichi Software,NaN,NaN,NaN,NaN,NaN,NaN,2020-07-30,2020-05-09
64013,"Yoru, Tomosu",NS,Visual Novel,Nippon Ichi Software,Nippon Ichi Software,NaN,NaN,NaN,NaN,NaN,NaN,2020-07-30,2020-05-09
64014,Yunohana SpRING! ~Mellow Times~,NS,Visual Novel,Idea Factory,Otomate,NaN,NaN,NaN,NaN,NaN,NaN,2019-02-28,2019-02-24
